<a href="https://colab.research.google.com/github/rklepov/hse-cs-ml-2018-2019/blob/homework/03-ML-basic/HW/hw5/hw5_clusterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# _All the news_

https://www.kaggle.com/snapcrack/all-the-news

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sklearn
sklearn.__version__

'0.21.2'

## 1. _Data preparation_

_"Проверьте данные на наличие аномалий"_ - я не очень понял, что здесь имеется в виду.

In [0]:
articles = pd.DataFrame()

for i in range(3):
    articles = articles.append(pd.read_csv('articles{}.csv'.format(1+i), index_col=0))

In [4]:
articles.tail()

,id,title,publication,author,date,year,month,url,content
146028,218078,An eavesdropping Uber driver saved his 16-year...,Washington Post,Avi Selk,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Uber driver Keith Avila picked up a p...
146029,218079,Plane carrying six people returning from a Cav...,Washington Post,Sarah Larimer,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Crews on Friday continued to search L...
146030,218080,After helping a fraction of homeowners expecte...,Washington Post,Renae Merle,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,When the Obama administration announced a...
146031,218081,"Yes, this is real: Michigan just banned bannin...",Washington Post,Chelsea Harvey,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,This story has been updated. A new law in...
146032,218082,What happened in Washington state after voters...,Washington Post,Christopher Ingraham,2016-12-29,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,The nation’s first recreational marijuana...


In [5]:
articles.shape

(142570, 9)

In [0]:
articles = articles[['title', 'content']]

In [7]:
articles.isnull().sum()

title      2
content    0
dtype: int64

In [0]:
articles['title'][articles['title'].isnull()] = ''

In [0]:
titles = articles['title']

texts = articles['content']

## 2. _Clusterization_ 

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation

from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

from sklearn import metrics

Ограничим размеры словарей (для сокращения времени работы алгоритмов).

In [0]:
titles_sparse = TfidfVectorizer(max_features=4096).fit_transform(titles)
texts_sparse = TfidfVectorizer(max_features=4096).fit_transform(texts)

In [12]:
titles_sparse.shape, texts_sparse.shape

((142570, 4096), (142570, 4096))

In [0]:
def get_title_metrics(X_train, _, y_pred):
    return { 'Silhouette' : metrics.silhouette_score(X_train, y_pred) }

def get_content_metrics(X_train, y_test, y_pred):
    result = {
        'ARI': metrics.adjusted_rand_score(y_test, y_pred),
        'AMI': metrics.adjusted_mutual_info_score(y_test, y_pred, average_method='arithmetic'),
        'Homogenity': metrics.homogeneity_score(y_test, y_pred),
        'Completeness': metrics.completeness_score(y_test, y_pred),
        'V-measure': metrics.v_measure_score(y_test, y_pred)}
    result.update(get_title_metrics(X_train, y_test, y_pred))
    return result

def print_metrics(get_metrics, dataset_name, algo_name, X_train, y_test, y_pred):
    print('{} for {}:'.format(algo_name, dataset_name))
    for name, val in get_metrics(X_train, y_test, y_pred).items():
        print('{}: {:.3f}'.format(name, val))
    print(40 * '*')

def print_title_metrics(algo_name, X_train, y_pred):
    print_metrics(get_title_metrics, 'titles', algo_name, X_train, None, y_pred)

def print_content_metrics(algo_name, X_train, y_test, y_pred):
    print_metrics(get_content_metrics, 'content', algo_name, X_train, y_test, y_pred)

### 2.1 _AffinityPropagation_

У меня **AffinityPropagation** не работает на большом датасете - не хватает памяти ([`MemoryError`](https://docs.python.org/3/library/exceptions.html)), пришлось его сильно уменьшить.

In [0]:
ix = np.random.choice(titles_sparse.shape[0], 2**13, replace=False)

X_train = titles_sparse[ix]
X_test = texts_sparse[ix]

In [15]:
%%time

y_test = AffinityPropagation().fit_predict(X_train)

print_title_metrics('AffinityPropagation', X_train, y_test)

AffinityPropagation for titles:
Silhouette: -0.038
****************************************
CPU times: user 4min 33s, sys: 3.55 s, total: 4min 37s
Wall time: 4min 37s


In [16]:
%%time

print_content_metrics('AffinityPropagation', X_train, y_test, AffinityPropagation().fit_predict(X_test))

AffinityPropagation for content:
ARI: -0.000
AMI: 0.003
Homogenity: 0.133
Completeness: 0.081
V-measure: 0.101
Silhouette: -0.030
****************************************
CPU times: user 5min, sys: 536 ms, total: 5min
Wall time: 5min


### 2.2 _DBSCAN_

Также пришлось уменьшить датасет ($1/5$ от исходных данных), а то долго считает.

In [0]:
_, X_train, _, X_test = train_test_split(titles_sparse, texts_sparse, test_size=0.2, random_state=299)

In [18]:
X_train.shape, X_test.shape

((28514, 4096), (28514, 4096))

In [19]:
%%time

y_test = DBSCAN().fit_predict(X_train)

print_title_metrics('DBSCAN', X_train, y_test)

DBSCAN for titles:
Silhouette: -0.284
****************************************
CPU times: user 28.4 s, sys: 156 ms, total: 28.6 s
Wall time: 28.6 s


In [20]:
%%time

print_content_metrics('DBSCAN', X_train, y_test, DBSCAN().fit_predict(X_test))

DBSCAN for content:
ARI: -0.003
AMI: -0.001
Homogenity: 0.000
Completeness: 0.000
V-measure: 0.000
Silhouette: -0.009
****************************************
CPU times: user 4min 31s, sys: 272 ms, total: 4min 32s
Wall time: 4min 32s


# 3. _PCA Clusterization_ 

In [21]:
titles_pca = TruncatedSVD(n_components=128).fit_transform(titles_sparse)
texts_pca = TruncatedSVD(n_components=128).fit_transform(texts_sparse)

titles_pca.shape, texts_pca.shape

((142570, 128), (142570, 128))

### 3.1 _AffinityPropagation_

In [0]:
X_train = titles_pca[ix]
X_test = texts_pca[ix]

In [23]:
%%time

y_test = AffinityPropagation().fit_predict(X_train)

print_title_metrics('PCA / AffinityPropagation', X_train, y_test)

PCA / AffinityPropagation for titles:
Silhouette: 0.009
****************************************
CPU times: user 1min 49s, sys: 251 ms, total: 1min 50s
Wall time: 1min 49s


In [24]:
%%time

print_content_metrics('PCA / AffinityPropagation', X_train, y_test, AffinityPropagation().fit_predict(X_test))

PCA / AffinityPropagation for content:
ARI: 0.011
AMI: 0.053
Homogenity: 0.471
Completeness: 0.452
V-measure: 0.462
Silhouette: -0.157
****************************************
CPU times: user 1min 24s, sys: 261 ms, total: 1min 24s
Wall time: 1min 23s


### 3.2 _DBSCAN_

In [0]:
_, X_train, _, X_test = train_test_split(titles_pca, texts_pca, test_size=0.2, random_state=299)

In [26]:
%%time

y_test = DBSCAN().fit_predict(X_train)

print_title_metrics('PCA / DBSCAN', X_train, y_test)

PCA / DBSCAN for titles:
Silhouette: 0.265
****************************************
CPU times: user 8min 29s, sys: 912 ms, total: 8min 30s
Wall time: 8min 22s


In [27]:
%%time

print_content_metrics('PCA / DBSCAN', X_train, y_test, DBSCAN().fit_predict(X_test))

PCA / DBSCAN for content:
ARI: -0.001
AMI: -0.000
Homogenity: 0.000
Completeness: 0.004
V-measure: 0.000
Silhouette: 0.099
****************************************
CPU times: user 8min 15s, sys: 771 ms, total: 8min 16s
Wall time: 8min 8s


### 3.3 _AgglomerativeClustering_

In [28]:
%%time

y_test = AgglomerativeClustering().fit_predict(X_train)

print_title_metrics('PCA / AgglomerativeClustering', X_train, y_test)

PCA / AgglomerativeClustering for titles:
Silhouette: 0.115
****************************************
CPU times: user 2min 15s, sys: 8.25 s, total: 2min 24s
Wall time: 2min 15s


In [29]:
%%time

print_content_metrics('PCA / AgglomerativeClustering', X_train, y_test, AgglomerativeClustering().fit_predict(X_test))

PCA / AgglomerativeClustering for content:
ARI: 0.034
AMI: 0.009
Homogenity: 0.017
Completeness: 0.006
V-measure: 0.009
Silhouette: 0.030
****************************************
CPU times: user 2min 8s, sys: 912 ms, total: 2min 9s
Wall time: 2min
